## Clustering Neighborhoods in Toronto - Part 1 getting data from Wikipedia

In [1]:
Import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
import urllib.request

In [3]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [6]:
page = urllib.request.urlopen(url)

In [7]:
# import the BeautifulSoup library so we can parse HTML and XML documents
! pip install BeautifulSoup4
from bs4 import BeautifulSoup
# parse the HTML from our URL into the BeautifulSoup parse tree format
! pip install lxml
! pip install bs4
! pip install html5lib

     |████████████████████████████████| 122kB 21.4MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 17.5MB/s eta 0:00:01     |█████▉                          | 1.0MB 17.5MB/s eta 0:00:01     |████████████████▏               | 2.8MB 17.5MB/s eta 0:00:01     |████████████████████▏           | 3.5MB 17.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [8]:
data = BeautifulSoup(page, "html5lib")

In [9]:
data.title
data.title.string

'List of postal codes of Canada: M - Wikipedia'

In [10]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=data.find_all("table")
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B
 </td>
 <td>Not assigned
 </td>
 <td>
 </td></tr>
 <tr>
 <t

In [11]:
right_table=data.find('table', class_='wikitable sortable')

In [12]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
import pandas as pd
df=pd.DataFrame(A,columns=['Postal_Code'])
df['Borough']=B
df['Neighborhood']=C

df.head()

,Postal_Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [13]:
#removing \n which is appearing extra at the end of every value
df2=df.replace({'\n': ''}, regex=True)

#removing values where Borough is Not assigned 
Df3= df2[df2['Borough']!='Not assigned']
Df3.head()

,Postal_Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
Df3['Neighborhood'] = np.where(Df3['Neighborhood'] == 'Parkwoods', Df3['Borough'], Df3['Neighborhood'])
Df3.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Postal_Code,Borough,Neighborhood
2,M3A,North York,North York
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
Df3.shape

(103, 3)

In [16]:
import pandas as pd
import io
import requests

url = "http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
s = requests.get(url).content
ds = pd.read_csv(io.StringIO(s.decode('utf-8')))
print(ds.describe())


         Latitude   Longitude
count  103.000000  103.000000
mean    43.704608  -79.397153
std      0.052463    0.097146
min     43.602414  -79.615819
25%     43.660567  -79.464763
50%     43.696948  -79.388790
75%     43.745320  -79.340923
max     43.836125  -79.160497


In [17]:
new_df = pd.merge(left=Df3, right=ds, left_on='Postal_Code', right_on='Postal Code')
Df4=new_df[['Postal_Code','Borough','Neighborhood','Latitude','Longitude']]
Df4.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,North York,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [20]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Df4['Latitude'], Df4['Longitude'], Df4['Borough'], Df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
